<h4>Autoencoder</h4>

<h5>Imports</h5>

In [1]:
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, unquote_plus
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, Dropout, Conv1D
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import jsonlines
import os
import json
import pickle
from sklearn.metrics import f1_score, accuracy_score, precision_score
from tensorflow.python.profiler import profiler_v2 as profiler

2023-05-16 19:52:36.537767: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-16 19:52:36.580711: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 19:52:37.159378: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_names=['malicious','normal']
for name in file_names:
    data=[]
    with jsonlines.open('dataset/'+name+'.txt') as reader:
        for line in reader:
            if line['request']['method'] == 'POST':
                data.append({'request':{'method':'POST', 'uri':line['request']['uri'], 'body':line['request']['body'],'headers':line['request']['headers']},'metadata':line['metdata']})
            else:
                data.append({'request':{'method':'GET', 'uri':line['request']['uri'], 'headers':line['request']['headers']},'metadata':line['metdata']})
    with jsonlines.open('dataset/'+name+'_clean.txt', mode='w') as writer:
        writer.write_all(data)

<h4>Load and create datasets</h4>

In [3]:
file_names = ['dataset/malicious_clean.txt' ,'dataset/normal_clean.txt']
data={}
for file in file_names:
    data[file]=[]
    with jsonlines.open(file) as reader:
        for line in reader:
            if line['request']['method'] == 'POST':
                data[file].append('POST'+' '+line['request']['uri']+' '+line['request']['body']+' '+json.dumps(line['request']['headers']))
            else:
                data[file].append('GET'+' '+line['request']['uri']+' '+json.dumps(line['request']['headers']))
                
                
normal = data[file_names[1]]
malicious = data[file_names[0]]
normal_part1 = normal[180000:]
normal_part2 = normal[:180000]

train_examples = normal_part2
test_examples = normal_part1+malicious
train_labels = [0] * len(train_examples)
test_labels = [0]* len(normal_part1)
test_labels.extend([1] * len(malicious))

dataset_train = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),train_examples)), train_labels))
dataset_test = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),test_examples)), test_labels))


def preprocess_text(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    text = tf.strings.regex_replace(text, '(.)',  r'\1 ')
    return text, label

def preprocess_text_substitution(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    punctuation = "=?/(){}[]<>"
    for p in punctuation:
        text = tf.strings.regex_replace(text, "\\" + p, " "+p+" ")
    return text, label

 
# Map the preprocess function to the dataset
train = dataset_train.map(preprocess_text)
test = dataset_test.map(preprocess_text)

train_sub = dataset_train.map(preprocess_text_substitution)
test_sub = dataset_test.map(preprocess_text_substitution)

for text, label in train_sub.take(1):
    print(text.numpy(), label.numpy())

for text, label in train.take(1):
    print(text.numpy(), label.numpy())

2023-05-16 19:52:52.281797: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 19:52:52.328630: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 19:52:52.328677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 19:52:52.330160: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 19:52:52.330216: I tensorflow/compile

b'a  / a / a / a  { "a" = "a" }   { "a-a": "a", "a": "a / a,* / *", "a-a": "a, a, a", "a-a": "a / n.n  ( a a n.n; x; x )  a / n.n  ( a, a a )  a / n.n.n.n a / n.n", "a": "a = x; a = x" } ' 0
b'a   / a / a / a   { " a " = " a " }   { " a - a " :   " a " ,   " a " :   " a / a , * / * " ,   " a - a " :   " a ,   a ,   a " ,   " a - a " :   " a / n . n   ( a   a   n . n ;   x ;   x )   a / n . n   ( a ,   a   a )   a / n . n . n . n   a / n . n " ,   " a " :   " a = x ;   a = x " } ' 0


2023-05-16 19:52:54.282347: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]
2023-05-16 19:52:54.302591: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]


<h4>Save and load vectorization from disk</h4>

In [4]:
#pickle.dump({'config': vectorization.get_config(),'weights': vectorization.get_weights()}, open("vectorization.pkl", "wb"))

#pickle.dump({'config': vectorization_sub.get_config(), 'weights': vectorization_sub.get_weights()}, open("vectorization_sub.pkl", "wb"))


from_disk = pickle.load(open("vectorization.pkl", "rb"))
vectorization = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization.set_weights(from_disk['weights'])

from_disk_sub = pickle.load(open("vectorization_sub.pkl", "rb"))
vectorization_sub = TextVectorization.from_config(from_disk_sub['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization_sub.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization_sub.set_weights(from_disk_sub['weights'])

2023-05-16 19:52:54.351758: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-05-16 19:52:54.480589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


<h4>Autoencoder</h4>

In [5]:
embedding_layer = tf.keras.layers.Embedding(1000, 4)
flatten_layer = tf.keras.layers.Flatten()

def autoencoder_processing(text, label):
    return  vectorization(text), vectorization(text)

def sub_processing(text, label):
    return  vectorization_sub(text), vectorization_sub(text)

def processing(text, label):
    return  vectorization(text), vectorization(text)

train_auto_sub = train_sub.map(sub_processing)
train_auto = train.map(processing)

train_auto = train_auto.batch(32)
train_auto_sub = train_auto_sub.batch(32)

input_layer = tf.keras.layers.Input(shape=(256,))
encoder = tf.keras.layers.Dense(256, activation='relu')(input_layer)
encoder = tf.keras.layers.Dense(128, activation='relu')(encoder)
encoder = tf.keras.layers.Dense(64, activation='relu')(encoder)
decoder = tf.keras.layers.Dense(128, activation='relu')(encoder)
decoder = tf.keras.layers.Dense(256, activation='relu')(decoder)
autoencoder = tf.keras.models.Model(input_layer, decoder)

input_layer_sub = tf.keras.layers.Input(shape=(128,))
encoder_sub = tf.keras.layers.Dense(128, activation='relu')(input_layer_sub)
encoder_sub = tf.keras.layers.Dense(64, activation='relu')(encoder_sub)
encoder_sub = tf.keras.layers.Dense(32, activation='relu')(encoder_sub)
decoder_sub = tf.keras.layers.Dense(64, activation='relu')(encoder_sub)
decoder_sub = tf.keras.layers.Dense(128, activation='relu')(decoder_sub)
autoencoder_sub = tf.keras.models.Model(input_layer_sub, decoder_sub)

# Compile and train the model
autoencoder.compile(optimizer='adam',  loss=losses.MeanSquaredError())
autoencoder_sub.compile(optimizer='adam',  loss=losses.MeanSquaredError())

import time
start_time = time.time()
autoencoder.fit(train_auto, epochs=10)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")
start_time = time.time()
autoencoder_sub.fit(train_auto_sub, epochs=10)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Epoch 1/10


2023-05-16 19:52:54.861358: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int64
	 [[{{node Placeholder/_7}}]]
2023-05-16 19:52:56.187147: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-16 19:52:56.190603: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0xc099b80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-16 19:52:56.190632: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-05-16 19:52:56.194845: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env v

5625/5625 [==============================] - 43s 7ms/step - loss: 4.3262
Epoch 2/10
5625/5625 [==============================] - 40s 7ms/step - loss: 3.1634
Epoch 3/10
5625/5625 [==============================] - 40s 7ms/step - loss: 2.9445
Epoch 4/10
5625/5625 [==============================] - 40s 7ms/step - loss: 2.8225
Epoch 5/10
5625/5625 [==============================] - 40s 7ms/step - loss: 2.7358
Epoch 6/10
5625/5625 [==============================] - 41s 7ms/step - loss: 2.6729
Epoch 7/10
5625/5625 [==============================] - 40s 7ms/step - loss: 2.6250
Epoch 8/10
5625/5625 [==============================] - 40s 7ms/step - loss: 2.5885
Epoch 9/10
5625/5625 [==============================] - 40s 7ms/step - loss: 2.5604
Epoch 10/10
5625/5625 [==============================] - 40s 7ms/step - loss: 2.5378
Elapsed time: 402.99103808403015 seconds
Epoch 1/10


2023-05-16 19:59:37.852001: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int64
	 [[{{node Placeholder/_7}}]]


5625/5625 [==============================] - 39s 7ms/step - loss: 93.9164
Epoch 2/10
5625/5625 [==============================] - 38s 7ms/step - loss: 58.1716
Epoch 3/10
5625/5625 [==============================] - 38s 7ms/step - loss: 52.8472
Epoch 4/10
5625/5625 [==============================] - 38s 7ms/step - loss: 50.2034
Epoch 5/10
5625/5625 [==============================] - 38s 7ms/step - loss: 47.0392
Epoch 6/10
5625/5625 [==============================] - 38s 7ms/step - loss: 45.1486
Epoch 7/10
5625/5625 [==============================] - 38s 7ms/step - loss: 41.8873
Epoch 8/10
5625/5625 [==============================] - 38s 7ms/step - loss: 39.9949
Epoch 9/10
5625/5625 [==============================] - 38s 7ms/step - loss: 40.1368
Epoch 10/10
5625/5625 [==============================] - 38s 7ms/step - loss: 39.2019
Elapsed time: 379.51816868782043 seconds


In [6]:
#autoencoder.fit(train_auto, epochs=10)
autoencoder.save('autoencoder/my_model')
autoencoder_sub.save('autoencoder_sub/my_model')

autoencoder = tf.keras.models.load_model('autoencoder/my_model')
autoencoder_sub = tf.keras.models.load_model('autoencoder_sub/my_model')


INFO:tensorflow:Assets written to: autoencoder/my_model/assets
INFO:tensorflow:Assets written to: autoencoder_sub/my_model/assets


In [38]:
def autoencoder_processing(text, label):
    return  vectorization(text), label
def autoencoder_processing_sub(text, label):
    return  vectorization_sub(text), label

test_auto = test.map(autoencoder_processing)
test_auto = test_auto.batch(32)
test_auto_sub = test_sub.map(autoencoder_processing_sub)
test_auto_sub = test_auto_sub.batch(32)

reconsturctions =  autoencoder.predict(test_auto)
reconsturctions_sub =  autoencoder_sub.predict(test_auto_sub)

   8/3685 [..............................] - ETA: 30s 

2023-04-23 01:10:41.067879: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype string
	 [[{{node Placeholder/_6}}]]


3685/3685 [==============================] - 24s 6ms/step


In [39]:
def get_data(x, y):
  return x

# Use the map() function to extract the labels
dataset_tmp = test_auto.map(get_data)
dataset_tmp_sub = test_auto_sub.map(get_data)

# Convert the labels dataset to a NumPy array
tmp = np.concatenate(list(dataset_tmp.as_numpy_iterator()))
mse = tf.keras.losses.mean_squared_error(tmp, reconsturctions)

tmp_sub = np.concatenate(list(dataset_tmp_sub.as_numpy_iterator()))
mse_sub = tf.keras.losses.mean_squared_error(tmp_sub, reconsturctions_sub)

2023-04-23 01:11:06.176232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [117899]
	 [[{{node Placeholder/_0}}]]


In [45]:
pred = list(map(lambda x: 0 if x.numpy()<0.09 else 1, list(mse)))
pred_sub = list(map(lambda x: 0 if x.numpy()<0.09 else 1, list(mse_sub)))

In [59]:
actual =test_auto.map(lambda x,y:y)
actual = np.concatenate(list(actual.as_numpy_iterator()))


2023-04-23 01:28:53.801071: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype string
	 [[{{node Placeholder/_6}}]]


In [62]:
print("Metrics")
print("f1: ", f1_score(actual,pred))
print("acc: ", accuracy_score(actual,pred))
print("prec: ", precision_score(actual,pred))

0.8628199463933316

In [ ]:
print("Metrics sub")
print("f1: ", f1_score(actual,pred_sub))
print("acc: ", accuracy_score(actual,pred_sub))
print("prec: ", precision_score(actual,pred_sub))